#Librerias

In [ ]:
import numpy as np
import scipy.spatial
import pandas as pd
from sklearn.metrics import davies_bouldin_score as DB
from sklearn.metrics.pairwise import euclidean_distances
import copy
from sklearn.metrics import adjusted_rand_score as ari

#Codigo

In [ ]:
class individuo:
  def __init__(self,*args,hijo = False):
    if hijo:
      self.umbrales =np.array([]);self.centroides=np.array([]);
    else:
      k_max,x_min,x_max,d = args
      self.umbrales =np.random.random(k_max)
      self.checar_umbrales(k_max)
      self.centroides=np.random.uniform(
          low=x_min,high=x_max,size=(k_max,d))
    self.aptitud = None
    self.fx = None
    self.particion = None
  def checar_umbrales(self,k_max):
    if np.count_nonzero(self.umbrales >= 0.5) < 2:
      indices = np.random.choice(range(k_max),2,replace = False)
      self.umbrales[indices] = np.random.uniform(0.5,1,2)
    self.activos = [i for i in range(k_max)
              if self.umbrales[i] > 0.5 ]

In [ ]:
def cruza_mutacion(Zk,Zi,Zj,Zm,F:float,CR):
  '''crea un individuo hijo con la cuza basada en 4 padres
  por lo que se aplica la mutacion al mismo tiempo que la mutacion
  F es un escalar que varia en cada generacion***
  CR es la constante de cruza'''
  k_max = len(Zk.umbrales); d = len(Zk.centroides[0])
  hijo = individuo(hijo=True)
  #cruzamos los centroides de los padres
  hijo.centroides = np.array(
    [Zk.centroides[i][j] + F*(Zi.centroides[i][j]-Zj.centroides[i][j])
    if np.random.random()<CR else Zm.centroides[i][j]
    for i in range(k_max) for j in range(d)]).reshape(k_max,d)
  #cruzamos los umbrales
  hijo.umbrales = np.array(
    [Zk.umbrales[i] + F*(Zi.umbrales[i]-Zj.umbrales[i])
     if np.random.random() <CR
     else Zm.umbrales[i] for i in range(k_max)] )
  #revisamos que los umbrales no salgan de su rango
  hijo.umbrales = np.array(
      [umbral if umbral<1 and umbral>0
       else 1 if umbral>1
       else 0.0 for umbral in hijo.umbrales ])
  #si hay menos de 2 umbrales activos escogemos 2 al azar y los activamos
  hijo.checar_umbrales(k_max)
  return(hijo)

In [ ]:
def asignar_datos(individuo, datos):
  ''' Asignamos los datos a cada grupo, si alguno queda
  con menos de 2 elementos se diveiden los datos entre
  los grupos activos, despues se mueven los centoides '''
  Gactivos = individuo.centroides[individuo.activos]
  distancias = euclidean_distances(datos,Gactivos)
  particion = np.argmin(distancias, axis = 1)
  #revisar que todos los grupos activos tengan al menos 2 puntos
  grupos,NelementosG = np.unique(particion,return_counts=True)
  if np.count_nonzero(NelementosG < 2) or len(grupos) < len(Gactivos):
    particion = repartir_datos(Gactivos,datos,distancias)
  #movemos solo los centroides activos al centro de sus elementos
  for i,g in enumerate(individuo.activos):
    individuo.centroides[g]=np.mean(
        datos[np.where(particion==i)[0].tolist()],axis=0)
  return(np.array(particion))

In [ ]:
def repartir_datos(activos,datos,distancias):
  '''divide los datos entre los centros, a cada centro le tocan
  los k datos mas cercanos a el, con k = #datos/#centros
  si el numero de datos no es multiplo del numero de centros, al
  ultimo centro de la lista le tocan los restantes'''
  nCentros = len(activos); nDatos = len(datos)
  asignados = np.zeros(nCentros); gDisponibles = list(range(nCentros))
  particion = np.zeros(nDatos); nDatosC = nDatos//nCentros
  for i,dato in enumerate(datos):
    cMasCercano = np.argmin(distancias[i])
    particion[i] = cMasCercano
    asignados[cMasCercano] +=1
    if i >= nDatosC-1:
      if nDatosC in asignados[gDisponibles]:
        igLleno = np.where(asignados[gDisponibles] ==nDatosC)[0][0]
        distancias[:,gDisponibles[igLleno]] = np.inf
        gDisponibles.remove(gDisponibles[igLleno])
  return(particion)

In [ ]:
def Cs(particion,activos,datos):
  '''particion es una lista con la etiquta de los datos,
  activos es la lista de los centroides de los grupos,
  matriz distancias es una variable global con la distancia entre
  todos los puntos del conjunto de datos'''
  global matriz_distancias
  dividendo = 0
  divisor = 0
  #iteramos por grupos para sacar su distancia interna y entre grupos
  for i,centroide in enumerate(activos):
    intra = 0; elementos = np.where(particion==i)[0].tolist()
    nPuntosG = len(elementos)
    #sacamos la distancia interna del grupo
    intra = np.max(
    [matriz_distancias[dato][elementos] for dato in elementos]
        ,axis=1)
    dividendo += np.mean(intra)
    #sacamos la minima distancia entre grupos
    centroides_vecinos = np.array(
        [centro for iter,centro in enumerate(activos)if iter != i] )
    divisor += min(
      [np.linalg.norm(centroide-vecino) for vecino in centroides_vecinos])
  return(dividendo/divisor)

In [ ]:
def iniciar(k_max,x_min,x_max,d,tPoblacion,datos,index):
  poblacion = [individuo(k_max,x_min,x_max,d) for i in range(tPoblacion)]
  for indi in poblacion:
    asignar_aptitud(indi,datos,index)
  best = sorted(poblacion, key=lambda x: x.aptitud,reverse=True)[0]
  return(poblacion,best)

In [ ]:
def asignar_aptitud(individuo,datos,index):
    individuo.particion = asignar_datos(individuo,datos)
    activos = individuo.centroides[individuo.activos]
    #asignamos la aptitud
    if index == 'cs':
      individuo.fx = Cs(individuo.particion,activos,datos)
      individuo.aptitud = 1/ (individuo.fx+.0001)
    else index == 'DB':
      individuo.fx = DB(datos,individuo.particion)
      individuo.aptitud = 1/ (individuo.fx+.0001)


In [ ]:
def acde(cr_max: float,cr_min: float,tPoblacion: int,
         k_max: int,x_min: float,x_max: float,nLlamadas: int,
         datos, index='cs'):
  '''cr_max y cr_min son el maximo y minimo valor de la constante de cruza
  tPoblacion es el numero de individuos que componen a la poblacion
  k_max es el numero maximo de grupos a crear
  nLlamadas nos dice el maximo numero de llamadas a la funcion
  objetivo que puede realizar el programa, podemos saber cuantas
  veces itera el algoritmo al dividir este valor entre el tamaño
  de la poblacion, pues por cada individuo generado realiza una
  llamada a la funcion objetivo
  index nos dice si se quiere validar con el indice CS o DB'''
  global matriz_distancias
  d = len(datos[0]); max_iter = nLlamadas//tPoblacion
  matriz_distancias = scipy.spatial.distance_matrix(datos,datos,2)
  #iniciamos la poblacion inicial y el mejor individuo actual
  poblacion,best = iniciar(k_max,x_min,x_max,d,tPoblacion,datos,index)
  for iteracion in range(max_iter-1):
    #imprimir(poblacion,x_min,x_max)
    f = 0.5 * (1+np.random.uniform())
    cr = (cr_max - cr_min) * (max_iter-iteracion) / max_iter
    for indice,individuo in enumerate(poblacion):
      Zi,Zj,Zm = np.random.choice(poblacion,3,replace=False)
      hijo = cruza_mutacion(individuo,Zi,Zj,Zm,f,cr)
      asignar_aptitud(hijo,datos,index)
      if hijo.aptitud > individuo.aptitud:
        poblacion[indice] = hijo
      if hijo.aptitud > best.aptitud:
        best = hijo
  return(best)